# Cluster Classifier

## Использование

Запустите ячейки по порядку.

In [15]:
"""
Классификатор запросов на основе кластеризации
Определяет категорию (кластер) запроса пользователя

Использование:
    python cluster_classifier.py                    # Интерактивный режим
    python cluster_classifier.py --query "получить код"  # Один запрос
    python cluster_classifier.py --train            # Переобучить модель
"""



'\nКлассификатор запросов на основе кластеризации\nОпределяет категорию (кластер) запроса пользователя\n\nИспользование:\n    python cluster_classifier.py                    # Интерактивный режим\n    python cluster_classifier.py --query "получить код"  # Один запрос\n    python cluster_classifier.py --train            # Переобучить модель\n'

In [16]:
import os
import sys
import re
import joblib
import numpy as np
import pandas as pd
from pathlib import Path

# Настройка кодировки для Windows
if sys.platform == 'win32':
    try:
        if hasattr(sys.stdout, 'reconfigure'):
            sys.stdout.reconfigure(encoding='utf-8')
        if hasattr(sys.stderr, 'reconfigure'):
            sys.stderr.reconfigure(encoding='utf-8')
    except:
        pass

# Попытка импортировать sklearn
try:
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.cluster import KMeans
    from sklearn.metrics.pairwise import cosine_similarity
    from scipy.sparse import csr_matrix, load_npz
    HAS_SKLEARN = True
except ImportError:
    HAS_SKLEARN = False
    print("[ERROR] sklearn/scipy не установлены!")
    print("Установите: pip install scikit-learn scipy")
    sys.exit(1)



In [17]:
# НАСТРОЙКИ


In [18]:
VECTORIZED_DIR = "files/vectorized"
VECTORIZER_FILE = os.path.join(VECTORIZED_DIR, "vectorizer_tfidf.pkl")
VECTORS_FILE = os.path.join(VECTORIZED_DIR, "vectors_tfidf_sparse.npz")
METADATA_FILE = os.path.join(VECTORIZED_DIR, "metadata_tfidf.csv")
CLUSTER_MODEL_FILE = os.path.join(VECTORIZED_DIR, "cluster_model.pkl")
CLUSTER_INFO_FILE = os.path.join(VECTORIZED_DIR, "cluster_info.csv")

# Параметры кластеризации
N_CLUSTERS = 10  # Количество кластеров (можно настроить)
RANDOM_STATE = 42



In [19]:
# ФУНКЦИИ


In [20]:
def basic_clean(text):
    """Базовая очистка текста"""
    if not isinstance(text, str):
        return ""
    text = text.lower().strip()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text


def load_data():
    """Загрузка векторизованных данных и метаданных"""
    print("\n📂 Загрузка данных...")
    
    # Загрузка векторизатора
    if not os.path.exists(VECTORIZER_FILE):
        raise FileNotFoundError(f"Векторизатор не найден: {VECTORIZER_FILE}")
    
    try:
        vectorizer = joblib.load(VECTORIZER_FILE)
        print(f"   ✅ Векторизатор загружен")
    except Exception as e:
        raise Exception(f"Ошибка при загрузке векторизатора: {e}")
    
    # Загрузка векторов
    if not os.path.exists(VECTORS_FILE):
        raise FileNotFoundError(f"Векторы не найдены: {VECTORS_FILE}")
    
    # Загрузка sparse матрицы (сохранена в формате numpy)
    try:
        loaded = np.load(VECTORS_FILE, allow_pickle=True)
        vectors = csr_matrix((loaded['data'], loaded['indices'], loaded['indptr']), 
                            shape=loaded['shape'])
        print(f"   ✅ Векторы загружены: {vectors.shape}")
    except (KeyError, ValueError) as e:
        # Попытка загрузить как scipy sparse формат
        try:
            vectors = load_npz(VECTORS_FILE)
            print(f"   ✅ Векторы загружены (scipy формат): {vectors.shape}")
        except Exception as e2:
            raise Exception(f"Ошибка при загрузке векторов: {e}, {e2}")
    except Exception as e:
        raise Exception(f"Ошибка при загрузке векторов: {e}")
    
    # Загрузка метаданных
    if not os.path.exists(METADATA_FILE):
        raise FileNotFoundError(f"Метаданные не найдены: {METADATA_FILE}")
    
    try:
        metadata = pd.read_csv(METADATA_FILE)
        print(f"   ✅ Метаданные загружены: {len(metadata)} строк")
    except Exception as e:
        raise Exception(f"Ошибка при загрузке метаданных: {e}")
    
    return vectorizer, vectors, metadata



In [21]:
def train_cluster_model(vectors, metadata, n_clusters=None, save=True):
    """
    Обучение модели кластеризации
    
    Parameters:
    -----------
    vectors : sparse matrix
        Векторизованные тексты
    metadata : DataFrame
        Метаданные с текстами
    n_clusters : int, optional
        Количество кластеров (по умолчанию: N_CLUSTERS)
    save : bool
        Сохранять ли модель
    """
    if n_clusters is None:
        n_clusters = N_CLUSTERS
    
    print(f"\n🔢 Обучение модели кластеризации (K-Means, {n_clusters} кластеров)...")
    
    # Обучение K-Means
    kmeans = KMeans(n_clusters=n_clusters, random_state=RANDOM_STATE, n_init=10, verbose=0)
    cluster_labels = kmeans.fit_predict(vectors)
    
    print(f"   ✅ Модель обучена")
    
    # Анализ кластеров
    cluster_info = []
    feature_names = None
    
    # Попытка получить названия признаков из векторизатора
    try:
        # Загружаем векторизатор для получения названий признаков
        vectorizer = joblib.load(VECTORIZER_FILE)
        feature_names = vectorizer.get_feature_names_out()
    except:
        pass
    
    # Создание информации о кластерах
    for i in range(n_clusters):
        cluster_mask = cluster_labels == i
        cluster_size = np.sum(cluster_mask)
        cluster_texts = metadata[cluster_mask]['cleaned'].tolist() if 'cleaned' in metadata.columns else []
        
        # Определение ключевых слов кластера
        keywords = []
        if feature_names is not None:
            cluster_center = kmeans.cluster_centers_[i]
            # Находим топ-5 признаков с наибольшими весами
            top_indices = np.argsort(cluster_center)[-5:][::-1]
            keywords = [feature_names[idx] for idx in top_indices]
        
        # Примеры текстов из кластера
        examples = cluster_texts[:3] if cluster_texts else []
        
        cluster_info.append({
            'cluster_id': i,
            'size': cluster_size,
            'percentage': (cluster_size / len(cluster_labels)) * 100,
            'keywords': ', '.join(keywords) if keywords else 'N/A',
            'examples': ' | '.join(examples[:3]) if examples else 'N/A'
        })
    
    cluster_df = pd.DataFrame(cluster_info)
    
    if save:
        # Сохранение модели
        joblib.dump(kmeans, CLUSTER_MODEL_FILE)
        print(f"   💾 Модель сохранена: {CLUSTER_MODEL_FILE}")
        
        # Сохранение информации о кластерах
        cluster_df.to_csv(CLUSTER_INFO_FILE, index=False, encoding='utf-8')
        print(f"   💾 Информация о кластерах сохранена: {CLUSTER_INFO_FILE}")
    
    return kmeans, cluster_df, cluster_labels



In [22]:
def load_cluster_model():
    """Загрузка обученной модели кластеризации"""
    if not os.path.exists(CLUSTER_MODEL_FILE):
        return None, None
    
    try:
        kmeans = joblib.load(CLUSTER_MODEL_FILE)
        cluster_info = pd.read_csv(CLUSTER_INFO_FILE) if os.path.exists(CLUSTER_INFO_FILE) else None
        return kmeans, cluster_info
    except Exception as e:
        print(f"[WARN] Ошибка при загрузке модели: {e}")
        return None, None



In [23]:
def predict_cluster(query, vectorizer, kmeans_model):
    """
    Предсказание кластера для запроса
    
    Parameters:
    -----------
    query : str
        Текст запроса
    vectorizer : TfidfVectorizer
        Векторизатор
    kmeans_model : KMeans
        Обученная модель кластеризации
        
    Returns:
    --------
    dict : Результат предсказания
    """
    # Очистка текста
    query_clean = basic_clean(query)
    
    if not query_clean:
        return {
            'cluster_id': None,
            'confidence': 0.0,
            'error': 'Пустой запрос'
        }
    
    # Векторизация запроса
    query_vector = vectorizer.transform([query_clean])
    
    # Предсказание кластера
    cluster_id = kmeans_model.predict(query_vector)[0]
    
    # Вычисление расстояния до центра кластера (уверенность)
    cluster_center = kmeans_model.cluster_centers_[cluster_id]
    query_dense = query_vector.toarray()[0]
    
    # Косинусное сходство для уверенности
    similarity = cosine_similarity([query_dense], [cluster_center])[0][0]
    confidence = max(0.0, min(1.0, (similarity + 1) / 2))  # Нормализация к [0, 1]
    
    return {
        'cluster_id': int(cluster_id),
        'confidence': float(confidence),
        'query_cleaned': query_clean
    }



In [24]:
def format_cluster_result(result, cluster_info=None):
    """Форматирование результата для вывода"""
    if result.get('error'):
        return f"[ОШИБКА] {result['error']}"
    
    cluster_id = result['cluster_id']
    confidence = result['confidence']
    
    output = []
    output.append("=" * 60)
    output.append("РЕЗУЛЬТАТ КЛАССИФИКАЦИИ")
    output.append("=" * 60)
    output.append(f"Запрос: {result.get('query_cleaned', 'N/A')}")
    output.append(f"Кластер: {cluster_id}")
    output.append(f"Уверенность: {confidence*100:.1f}%")
    
    if cluster_info is not None and cluster_id < len(cluster_info):
        cluster_data = cluster_info.iloc[cluster_id]
        output.append("")
        output.append("Информация о кластере:")
        output.append(f"  Размер: {int(cluster_data['size'])} запросов ({cluster_data['percentage']:.1f}%)")
        output.append(f"  Ключевые слова: {cluster_data.get('keywords', 'N/A')}")
        if 'examples' in cluster_data and pd.notna(cluster_data['examples']):
            output.append(f"  Примеры: {cluster_data['examples']}")
    
    output.append("=" * 60)
    
    return '\n'.join(output)



In [25]:
def interactive_mode(vectorizer, kmeans_model, cluster_info=None):
    """Интерактивный режим работы"""
    print("=" * 60)
    print("КЛАССИФИКАТОР ЗАПРОСОВ ПО КЛАСТЕРАМ")
    print("=" * 60)
    print("Введите запрос для определения его кластера/категории")
    print("Для выхода введите: 'exit', 'quit', 'выход' или 'q'")
    print("=" * 60)
    print()
    
    while True:
        try:
            # Ввод запроса
            query = input("\n[ВВОД] Введите запрос: ").strip()
            
            # Проверка на выход
            if query.lower() in ['exit', 'quit', 'выход', 'q', '']:
                print("\n[INFO] Выход из программы...")
                break
            
            if not query:
                print("[WARN] Пустой запрос. Попробуйте еще раз.")
                continue
            
            # Предсказание
            print("\n[PROCESSING] Обработка запроса...")
            result = predict_cluster(query, vectorizer, kmeans_model)
            
            # Вывод результата
            print("\n" + format_cluster_result(result, cluster_info))
            
        except KeyboardInterrupt:
            print("\n\n[INFO] Прервано пользователем. Выход...")
            break
        except Exception as e:
            print(f"\n[ERROR] Ошибка при обработке: {e}")
            import traceback
            traceback.print_exc()


In [26]:
# ОСНОВНАЯ ПРОГРАММА


In [27]:
def main(query=None, train=False, clusters=None):
    """
    Основная функция
    
    Parameters:
    -----------
    query : str, optional
        Запрос для классификации (если не указан - интерактивный режим)
    train : bool, optional
        Переобучить модель кластеризации (по умолчанию: False)
    clusters : int, optional
        Количество кластеров (по умолчанию: N_CLUSTERS)
    """
    # Определяем, запущено ли в Jupyter notebook
    try:
        from IPython import get_ipython
        in_notebook = get_ipython() is not None
    except:
        in_notebook = False
    
    # Если не в notebook, используем argparse
    if not in_notebook:
        import argparse
        
        parser = argparse.ArgumentParser(
            description='Классификатор запросов на основе кластеризации',
            formatter_class=argparse.RawDescriptionHelpFormatter,
            epilog="""
Примеры использования:
  python cluster_classifier.py                    # Интерактивный режим
  python cluster_classifier.py --query "получить код"  # Один запрос
  python cluster_classifier.py --train            # Переобучить модель
  python cluster_classifier.py --train --clusters 15   # Обучить с 15 кластерами
            """
        )
        
        parser.add_argument('--query', '-q',
                           type=str,
                           default=None,
                           help='Запрос для классификации (если не указан - интерактивный режим)')
        
        parser.add_argument('--train', '-t',
                           action='store_true',
                           help='Переобучить модель кластеризации')
        
        parser.add_argument('--clusters', '-c',
                           type=int,
                           default=N_CLUSTERS,
                           help=f'Количество кластеров (по умолчанию: {N_CLUSTERS})')
        
        args = parser.parse_args()
        query = args.query
        train = args.train
        clusters = args.clusters if args.clusters != N_CLUSTERS else None
    
    # Используем переданные параметры или значения по умолчанию
    if clusters is None:
        clusters = N_CLUSTERS
    
    # Загрузка данных
    try:
        vectorizer, vectors, metadata = load_data()
    except Exception as e:
        print(f"[ERROR] Ошибка при загрузке данных: {e}")
        if not in_notebook:
            sys.exit(1)
        return
    
    # Загрузка или обучение модели
    kmeans_model, cluster_info = load_cluster_model()
    
    if train or kmeans_model is None:
        print("\n[INFO] Обучение новой модели...")
        kmeans_model, cluster_info, cluster_labels = train_cluster_model(
            vectors, metadata, n_clusters=clusters, save=True
        )
        print("\n[INFO] Модель обучена и сохранена!")
    else:
        print("\n[INFO] Использование сохраненной модели")
        if cluster_info is not None:
            print(f"   Найдено {len(cluster_info)} кластеров")
    
    # Режим работы
    if query:
        # Одиночный запрос
        print(f"\n[INPUT] Запрос: {query}")
        result = predict_cluster(query, vectorizer, kmeans_model)
        print("\n" + format_cluster_result(result, cluster_info))
    else:
        # Интерактивный режим
        interactive_mode(vectorizer, kmeans_model, cluster_info)


In [ ]:
# Для использования в notebook просто вызовите:
# main()                    # Интерактивный режим
# main(query="получить код")  # Один запрос
# main(train=True)          # Переобучить модель
# main(train=True, clusters=15)  # Обучить с 15 кластерами

# Для использования из командной строки (в .py файле):
if __name__ == "__main__":
    main()





📂 Загрузка данных...
   ✅ Векторизатор загружен
   ✅ Векторы загружены: (1000, 1234)
   ✅ Метаданные загружены: 1000 строк

[INFO] Использование сохраненной модели
   Найдено 10 кластеров
КЛАССИФИКАТОР ЗАПРОСОВ ПО КЛАСТЕРАМ
Введите запрос для определения его кластера/категории
Для выхода введите: 'exit', 'quit', 'выход' или 'q'


[PROCESSING] Обработка запроса...

РЕЗУЛЬТАТ КЛАССИФИКАЦИИ
Запрос: получение кода
Кластер: 5
Уверенность: 53.9%

Информация о кластере:
  Размер: 445 запросов (44.5%)
  Ключевые слова: не, посылки, посылку, заказа, код
  Примеры: получение куар хода | оператора можно связаться | мне пришла смска от получения заказа
